In [14]:
import numpy as np
import warnings 
import sys
import pandas as pd
import scipy
from scipy.io import arff
if not sys.warnoptions:
    warnings.simplefilter("ignore")
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [15]:
data, meta = scipy.io.arff.loadarff('mtr-datasets/andro.arff')
data = pd.DataFrame(data)
data.head()

,Window0-Att0,Window0-Att1,Window0-Att2,Window0-Att3,Window0-Att4,Window0-Att5,Window1-Att0,Window1-Att1,Window1-Att2,Window1-Att3,...,Window4-Att2,Window4-Att3,Window4-Att4,Window4-Att5,Target,Target_2,Target_3,Target_4,Target_5,Target_6
0,11.84,5.08,49.0,32.2,16.5,1.40,12.01,5.09,48.0,32.0,...,45.0,29.7,38.5,3.27,15.17,5.14,46.0,30.4,46.8,3.74
1,12.01,5.09,48.0,32.0,14.8,1.25,12.89,5.10,48.0,31.9,...,46.0,30.1,42.6,3.55,14.85,5.11,47.0,31.2,46.8,3.78
2,12.89,5.10,48.0,31.9,8.6,0.71,13.95,5.11,47.0,30.8,...,48.0,31.3,46.0,3.79,14.72,5.10,48.0,31.7,42.7,3.40
3,13.95,5.11,47.0,30.8,20.2,1.64,13.75,5.13,45.0,29.7,...,47.0,31.1,46.7,3.79,15.28,5.11,48.0,32.0,42.5,3.38
4,13.75,5.13,45.0,29.7,38.5,3.27,13.34,5.13,46.0,30.1,...,45.0,29.7,45.1,3.75,16.15,5.13,47.0,30.8,42.1,3.27


In [16]:
X = data.iloc[:,:-6].values
y = data.iloc[:,-6:].values

scaler = StandardScaler()

X = scaler.fit_transform(X)
y = scaler.fit_transform(y)

print(X.shape,y.shape)

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

(49, 30) (49, 6)


In [17]:
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(39, 30) (10, 30) (39, 6) (10, 6)


In [18]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,f1_score,r2_score
def evaluate(y_train,y_test):
    print('MAE - ',mean_absolute_error(y_train,y_test))
    print('RMSE - ',mean_squared_error(y_train,y_test)**0.5)
    print('R-squared - ',r2_score(y_train,y_test))
    
    r2 = r2_score(y_train,y_test)
    
    n = 49
    p = 30

    adj_r2 = 1-(1-r2)*(n-1)/(n-p-1)
    print('Adjusted R-squared - ',adj_r2)
    
    return r2

In [19]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout

In [20]:
def create_c1node(X_feed,y_now):
    model = Sequential()
    model.add(Dense(128,activation='relu',input_shape=(X_feed.shape[1],),kernel_initializer='he_normal'))
    model.add(Dense(64,activation='relu',kernel_initializer='he_normal'))
    model.add(Dense(y_now.shape[1],activation='linear',kernel_initializer='he_normal'))
    model.compile(loss='mse', optimizer='adam', metrics=['mse'])
    
    return model

In [21]:
from time import time

In [22]:
chain = []

X_feed = X_train.copy()
t1 = time()

for i in range(y_train.shape[1]):
    print('Training chain node ',i)
    y_now = y_train[:,[i,]].copy()
    print('Shapes\n X = {} \n y = {}'.format(X_feed.shape,y_now.shape))
    
    node = create_c1node(X_feed,y_now)
    node.fit(X_feed,y_now,epochs=500,batch_size=50)
    print('Training of node {} complete\n'.format(i))
    chain.append(node)
    X_feed = np.append(X_feed,y_now,axis=1)
    
t2 = time()

print('Time taken ',(t2-t1))

Training chain node  0
Shapes
 X = (39, 30) 
 y = (39, 1)
Epoch 1/500
39/39 [==============================] - 1s - loss: 0.6234 - mean_squared_error: 0.6234
Epoch 2/500
39/39 [==============================] - 0s - loss: 0.4519 - mean_squared_error: 0.4519
Epoch 3/500
39/39 [==============================] - 0s - loss: 0.3548 - mean_squared_error: 0.3548
Epoch 4/500
39/39 [==============================] - 0s - loss: 0.2878 - mean_squared_error: 0.2878
Epoch 5/500
39/39 [==============================] - 0s - loss: 0.2426 - mean_squared_error: 0.2426
Epoch 6/500
39/39 [==============================] - 0s - loss: 0.2103 - mean_squared_error: 0.2103
Epoch 7/500
39/39 [==============================] - 0s - loss: 0.1837 - mean_squared_error: 0.1837
Epoch 8/500
39/39 [==============================] - 0s - loss: 0.1575 - mean_squared_error: 0.1575
Epoch 9/500
39/39 [==============================] - 0s - loss: 0.1327 - mean_squared_error: 0.1327
Epoch 10/500
39/39 [======================

39/39 [==============================] - 0s - loss: 1.0330e-04 - mean_squared_error: 1.0330e-04
Epoch 80/500
39/39 [==============================] - 0s - loss: 9.7012e-05 - mean_squared_error: 9.7012e-05
Epoch 81/500
39/39 [==============================] - 0s - loss: 9.0922e-05 - mean_squared_error: 9.0922e-05
Epoch 82/500
39/39 [==============================] - 0s - loss: 8.2577e-05 - mean_squared_error: 8.2577e-05
Epoch 83/500
39/39 [==============================] - 0s - loss: 7.2097e-05 - mean_squared_error: 7.2097e-05
Epoch 84/500
39/39 [==============================] - 0s - loss: 6.4024e-05 - mean_squared_error: 6.4024e-05
Epoch 85/500
39/39 [==============================] - 0s - loss: 5.8632e-05 - mean_squared_error: 5.8632e-05
Epoch 86/500
39/39 [==============================] - 0s - loss: 5.4649e-05 - mean_squared_error: 5.4649e-05
Epoch 87/500
39/39 [==============================] - 0s - loss: 5.0755e-05 - mean_squared_error: 5.0755e-05
Epoch 88/500
39/39 [============

39/39 [==============================] - 0s - loss: 8.4139e-08 - mean_squared_error: 8.4139e-08
Epoch 154/500
39/39 [==============================] - 0s - loss: 7.6084e-08 - mean_squared_error: 7.6084e-08
Epoch 155/500
39/39 [==============================] - 0s - loss: 6.7704e-08 - mean_squared_error: 6.7704e-08
Epoch 156/500
39/39 [==============================] - 0s - loss: 6.1294e-08 - mean_squared_error: 6.1294e-08
Epoch 157/500
39/39 [==============================] - 0s - loss: 5.7010e-08 - mean_squared_error: 5.7010e-08
Epoch 158/500
39/39 [==============================] - 0s - loss: 5.2150e-08 - mean_squared_error: 5.2150e-08
Epoch 159/500
39/39 [==============================] - 0s - loss: 4.6376e-08 - mean_squared_error: 4.6376e-08
Epoch 160/500
39/39 [==============================] - 0s - loss: 3.9649e-08 - mean_squared_error: 3.9649e-08
Epoch 161/500
39/39 [==============================] - 0s - loss: 3.3896e-08 - mean_squared_error: 3.3896e-08
Epoch 162/500
39/39 [===

39/39 [==============================] - 0s - loss: 1.3552e-11 - mean_squared_error: 1.3552e-11
Epoch 228/500
39/39 [==============================] - 0s - loss: 1.1878e-11 - mean_squared_error: 1.1878e-11
Epoch 229/500
39/39 [==============================] - 0s - loss: 1.0891e-11 - mean_squared_error: 1.0891e-11
Epoch 230/500
39/39 [==============================] - 0s - loss: 1.0921e-11 - mean_squared_error: 1.0921e-11
Epoch 231/500
39/39 [==============================] - 0s - loss: 9.7043e-12 - mean_squared_error: 9.7043e-12
Epoch 232/500
39/39 [==============================] - 0s - loss: 8.6782e-12 - mean_squared_error: 8.6782e-12
Epoch 233/500
39/39 [==============================] - 0s - loss: 7.7145e-12 - mean_squared_error: 7.7145e-12
Epoch 234/500
39/39 [==============================] - 0s - loss: 6.9033e-12 - mean_squared_error: 6.9033e-12
Epoch 235/500
39/39 [==============================] - 0s - loss: 6.2614e-12 - mean_squared_error: 6.2614e-12
Epoch 236/500
39/39 [===

39/39 [==============================] - 0s - loss: 1.3641e-14 - mean_squared_error: 1.3641e-14
Epoch 302/500
39/39 [==============================] - 0s - loss: 1.6762e-14 - mean_squared_error: 1.6762e-14
Epoch 303/500
39/39 [==============================] - 0s - loss: 1.1868e-14 - mean_squared_error: 1.1868e-14
Epoch 304/500
39/39 [==============================] - 0s - loss: 9.5600e-15 - mean_squared_error: 9.5600e-15
Epoch 305/500
39/39 [==============================] - 0s - loss: 1.3802e-14 - mean_squared_error: 1.3802e-14
Epoch 306/500
39/39 [==============================] - 0s - loss: 1.4459e-14 - mean_squared_error: 1.4459e-14
Epoch 307/500
39/39 [==============================] - 0s - loss: 1.0551e-14 - mean_squared_error: 1.0551e-14
Epoch 308/500
39/39 [==============================] - 0s - loss: 1.1662e-14 - mean_squared_error: 1.1662e-14
Epoch 309/500
39/39 [==============================] - 0s - loss: 8.1780e-15 - mean_squared_error: 8.1780e-15
Epoch 310/500
39/39 [===

...

39/39 [==============================] - 0s - loss: 2.3002e-05 - mean_squared_error: 2.3002e-05
Epoch 138/500
39/39 [==============================] - 0s - loss: 2.2026e-05 - mean_squared_error: 2.2026e-05
Epoch 139/500
39/39 [==============================] - 0s - loss: 2.1148e-05 - mean_squared_error: 2.1148e-05
Epoch 140/500
39/39 [==============================] - 0s - loss: 2.0344e-05 - mean_squared_error: 2.0344e-05
Epoch 141/500
39/39 [==============================] - 0s - loss: 1.9549e-05 - mean_squared_error: 1.9549e-05
Epoch 142/500
39/39 [==============================] - 0s - loss: 1.8743e-05 - mean_squared_error: 1.8743e-05
Epoch 143/500
39/39 [==============================] - 0s - loss: 1.7934e-05 - mean_squared_error: 1.7934e-05
Epoch 144/500
39/39 [==============================] - 0s - loss: 1.7154e-05 - mean_squared_error: 1.7154e-05
Epoch 145/500
39/39 [==============================] - 0s - loss: 1.6423e-05 - mean_squared_error: 1.6423e-05
Epoch 146/500
39/39 [===

39/39 [==============================] - 0s - loss: 1.2086e-06 - mean_squared_error: 1.2086e-06
Epoch 212/500
39/39 [==============================] - 0s - loss: 1.1625e-06 - mean_squared_error: 1.1625e-06
Epoch 213/500
39/39 [==============================] - 0s - loss: 1.1179e-06 - mean_squared_error: 1.1179e-06
Epoch 214/500
39/39 [==============================] - 0s - loss: 1.0745e-06 - mean_squared_error: 1.0745e-06
Epoch 215/500
39/39 [==============================] - 0s - loss: 1.0316e-06 - mean_squared_error: 1.0316e-06
Epoch 216/500
39/39 [==============================] - 0s - loss: 9.8936e-07 - mean_squared_error: 9.8936e-07
Epoch 217/500
39/39 [==============================] - 0s - loss: 9.4863e-07 - mean_squared_error: 9.4863e-07
Epoch 218/500
39/39 [==============================] - 0s - loss: 9.0890e-07 - mean_squared_error: 9.0890e-07
Epoch 219/500
39/39 [==============================] - 0s - loss: 8.7008e-07 - mean_squared_error: 8.7008e-07
Epoch 220/500
39/39 [===

39/39 [==============================] - 0s - loss: 4.4306e-08 - mean_squared_error: 4.4306e-08
Epoch 286/500
39/39 [==============================] - 0s - loss: 4.2143e-08 - mean_squared_error: 4.2143e-08
Epoch 287/500
39/39 [==============================] - 0s - loss: 4.0062e-08 - mean_squared_error: 4.0062e-08
Epoch 288/500
39/39 [==============================] - 0s - loss: 3.8148e-08 - mean_squared_error: 3.8148e-08
Epoch 289/500
39/39 [==============================] - 0s - loss: 3.6303e-08 - mean_squared_error: 3.6303e-08
Epoch 290/500
39/39 [==============================] - 0s - loss: 3.4550e-08 - mean_squared_error: 3.4550e-08
Epoch 291/500
39/39 [==============================] - 0s - loss: 3.2868e-08 - mean_squared_error: 3.2868e-08
Epoch 292/500
39/39 [==============================] - 0s - loss: 3.1203e-08 - mean_squared_error: 3.1203e-08
Epoch 293/500
39/39 [==============================] - 0s - loss: 2.9646e-08 - mean_squared_error: 2.9646e-08
Epoch 294/500
39/39 [===

39/39 [==============================] - 0s - loss: 7.5868e-10 - mean_squared_error: 7.5868e-10
Epoch 360/500
39/39 [==============================] - 0s - loss: 7.1223e-10 - mean_squared_error: 7.1223e-10
Epoch 361/500
39/39 [==============================] - 0s - loss: 6.7023e-10 - mean_squared_error: 6.7023e-10
Epoch 362/500
39/39 [==============================] - 0s - loss: 6.3129e-10 - mean_squared_error: 6.3129e-10
Epoch 363/500
39/39 [==============================] - 0s - loss: 5.9533e-10 - mean_squared_error: 5.9533e-10
Epoch 364/500
39/39 [==============================] - 0s - loss: 5.5638e-10 - mean_squared_error: 5.5638e-10
Epoch 365/500
39/39 [==============================] - 0s - loss: 5.2577e-10 - mean_squared_error: 5.2577e-10
Epoch 366/500
39/39 [==============================] - 0s - loss: 4.9187e-10 - mean_squared_error: 4.9187e-10
Epoch 367/500
39/39 [==============================] - 0s - loss: 4.6347e-10 - mean_squared_error: 4.6347e-10
Epoch 368/500
39/39 [===

39/39 [==============================] - 0s - loss: 6.2486e-12 - mean_squared_error: 6.2486e-12
Epoch 434/500
39/39 [==============================] - 0s - loss: 6.1213e-12 - mean_squared_error: 6.1213e-12
Epoch 435/500
39/39 [==============================] - 0s - loss: 5.6132e-12 - mean_squared_error: 5.6132e-12
Epoch 436/500
39/39 [==============================] - 0s - loss: 5.2926e-12 - mean_squared_error: 5.2926e-12
Epoch 437/500
39/39 [==============================] - 0s - loss: 5.0045e-12 - mean_squared_error: 5.0045e-12
Epoch 438/500
39/39 [==============================] - 0s - loss: 4.7298e-12 - mean_squared_error: 4.7298e-12
Epoch 439/500
39/39 [==============================] - 0s - loss: 4.3201e-12 - mean_squared_error: 4.3201e-12
Epoch 440/500
39/39 [==============================] - 0s - loss: 4.1877e-12 - mean_squared_error: 4.1877e-12
Epoch 441/500
39/39 [==============================] - 0s - loss: 4.0324e-12 - mean_squared_error: 4.0324e-12
Epoch 442/500
39/39 [===

In [23]:
y_pred = []

X_feed = X_test.copy()
i = 0
for node in chain:
    print('getting output from chain node ',i)
    node_name = 'Node' + str(i)
    print('Shapes - \n X = {}'.format(X_feed.shape))
    output = node.predict(X_feed)
    y_pred.append(output)
    X_feed = np.append(X_feed,output,axis=1)
    i += 1

getting output from chain node  0
Shapes - 
 X = (10, 30)
getting output from chain node  1
Shapes - 
 X = (10, 31)
getting output from chain node  2
Shapes - 
 X = (10, 32)
getting output from chain node  3
Shapes - 
 X = (10, 33)
getting output from chain node  4
Shapes - 
 X = (10, 34)
getting output from chain node  5
Shapes - 
 X = (10, 35)


In [24]:
predictions = np.array([0])
for i in y_pred:
    x = np.array(i)
    if predictions.shape == (1,):
        predictions = x
    else:
        predictions = np.append(predictions,x,axis=1)

In [25]:
r2 = evaluate(scaler.inverse_transform(y_test),scaler.inverse_transform(np.array(predictions)))

MAE -  2.2959100421269736
RMSE -  5.217795658540315
R-squared -  -0.7761905705102774
Adjusted R-squared -  -3.7365081880274067
